# Desafio Data Science Intelivix

In [271]:
# Dataset
# from sklearn.datasets import load_iris

# Classificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Pré-processamento
# from sklearn.preprocessing import StandardScaler

# Treinamento
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# Outros
import os
import pandas as pd
from itertools import count

# Dataset

In [272]:
# Easy way :)
# iris = load_iris()
# X, y = iris.data, iris.target

ds = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                 header=None,
                 names=['sl', 'sw', 'pl', 'pw', 'cl']
                )
# Mapeia as classes para números
class_map = { c:i for c, i in zip(ds['cl'].unique(), count()) }
ds['t'] = ds['cl'].map(class_map)

X, y = ds[['sl', 'sw', 'pl', 'pw']], ds['t']

# Classificadores

In [261]:
classifiers = [
    KNeighborsClassifier(),
    SVC(),
    LogisticRegression()
]

## Otimizando os parâmetros dos classificadores

In [262]:
# Definindo os valores possíveis dos hiperparâmetros
c_range = list(pow(10,n) for n in range(-4,4))
params = [
    { 'n_neighbors': list(range(1,20)), 'weights': ['uniform', 'distance'] },
    { 'kernel':('linear', 'rbf'), 'C': c_range },
    {'C': c_range }
]

# Aqui, cada classificador é testado em cross-validation pelo GridSearch.
# Os dados são divididos em 3, por padrão, e cada parâmetro é testado 3 vezes.
grids = [GridSearchCV(c, p, scoring='accuracy').fit(X, y) 
         for c, p in zip(classifiers, params)]

# Poderíamos aqui usar a propriedade 'best_estimator_' para recuperar
# os melhores classificadores em GridSearch, e ainda compará-los com a média de acertos,
# em 'best_score_'. Mas treinaremos novas instâncias para analisar as matrizes de confusão.
knn = KNeighborsClassifier(**grids[0].best_params_)
svc = SVC(**grids[1].best_params_)
lgr = LogisticRegression(**grids[2].best_params_)

classifiers = [knn, svc, lgr]
classifiers_names = ['KNN', 'SVC', 'L. Regression']

## Treinando os classificadores

In [275]:
# Dividindo as amostras em 66% treino e 34% testes
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=3)

predicted = []
for c, name in zip(classifiers, classifiers_names):
    c.fit(X_train, y_train)
    
    # Percentual de acertos no conjunto de testes
    predict = c.predict(X_test)
    predicted.append(predict)
    print("{0}: {1:.2%}".
          format(name, accuracy_score(y_test, predict)))

KNN: 94.12%
SVC: 96.08%
L. Regression: 92.16%


## Matrizes de confusão

In [276]:
for c, name, pred in zip(classifiers, classifiers_names, predicted):
    print('{0} - Matriz de confusão:'.format(name))
    print(confusion_matrix(y_test, pred))
    print()

KNN - Matriz de confusão:
[[19  0  0]
 [ 0 14  3]
 [ 0  0 15]]

SVC - Matriz de confusão:
[[19  0  0]
 [ 0 15  2]
 [ 0  0 15]]

L. Regression - Matriz de confusão:
[[19  0  0]
 [ 0 13  4]
 [ 0  0 15]]

